# QR Benchmark

In [2]:
import numpy as np
import time
from alc import *

#### Gram-Schmidt naif implementation

In [7]:
def QR_GS_naif(A, atol=1e-12, retorna_nops=False):
    """
    A una matriz de n x n
    atol la tolerancia con la que se filtran elementos nulos en R (no utilizado).
    retorna_nops permite (opcionalmente) retornar el número de operaciones realizadas
    Retorna matrices Q y R calculadas con Gram-Schmidt (y como tercer argumento opcional,
    el número de operaciones).
    Si la matriz A no es cuadrada, retorna None.
    """
    m, n = A.shape
    if m != n:
        return None # Solo matrices cuadradas
    
    operaciones = 0

    Q = np.zeros((m, n)) 
    Q[:, 0] = A[:, 0] / norma(A[:, 0], p=2)
    operaciones += m + 2*m # División elemento a elemento + cálculo de norma

    for col in range(1, n):
        Q[:, col] = A[:, col]
        for i in range(col):
            factor = producto_interno(Q[:, col], Q[:, i]) / producto_interno(Q[:, i], Q[:, i])
            operaciones += 2*(2*m-1) + 1  # Dos productos internos + una división

            Q[:, col] = Q[:, col] - factor * Q[:, i]
            operaciones += 2*m # Multiplicación y resta elemento a elemento

        Q[:, col] = Q[:, col] / norma(Q[:, col], p=2)
        operaciones += m + 2*m  # División elemento a elemento + cálculo de norma
    
    R = multiplicar(transpuesta(Q), A)
    operaciones += n**2 * (2*n)  # Producto matricial

    if retorna_nops:
        return Q, R, operaciones
    else:
        return Q, R

#### Gram-Schmidt efficient implementation

In [6]:
def QR_GS_efficient(A, atol=1e-12, retorna_nops=False):
    """
    A una matriz de n x n
    atol la tolerancia con la que se filtran elementos nulos en R (no utilizado).
    retorna_nops permite (opcionalmente) retornar el número de operaciones realizadas
    Retorna matrices Q y R calculadas con Gram-Schmidt (y como tercer argumento opcional,
    el número de operaciones).
    Si la matriz A no es cuadrada, retorna None.
    """
    m, n = A.shape
    if m < n:
        # raise ValueError
        return None # Solo matrices cuadradas
    
    operaciones = 0

    Q = np.zeros((m, n))
    R = np.zeros((m, n))

    # Iterar sobre las columnas de A
    for j in range(n):
        v = A[:, j].copy()

        # Ortogonalizar contra las columnas anteriores de Q
        for i in range(j):
            R[i, j] = producto_interno(v, Q[:, i])
            operaciones += 2*m - 1  # Producto interno

            v -= R[i, j] * Q[:, i]
            operaciones += 2*m  # Multiplicación y resta elemento a elemento

        R[j, j] = norma(v, p=2)
        Q[:, j] = v / R[j, j]
        operaciones += m + 2*m  # División elemento a elemento + cálculo de norma
        
    if retorna_nops:
        return Q, R, operaciones
    return Q, R

#### Householder naif implementation

In [10]:
def QR_HH_naif(A, atol=1e-12):
    """
    A una matriz de m x n (m >= n)
    atol la tolerancia con la que se filtran elementos nulos en R.
    Retorna matrices Q y R calculadas con reflexiones de Householder.
    Si la matriz A no cumple m >= n, retorna None.
    """
    m, n = A.shape
    if m < n:
        return None
    
    Q = np.eye(m)
    R = np.array(A, dtype=np.float64)

    for col in range(n):
        # Construyo vector u de la subcolumna
        u = np.zeros(m)
        for i in range(col, m):
            u[i] = R[i][col]
        norma_u = norma(u, p=2)
        if norma_u < atol:
            continue
        
        signo = np.sign(R[col][col]) if R[col][col] != 0 else 1
        e = np.zeros(m)
        e[col] = 1
        for i in range(m):
            u[i] += signo * norma_u * e[i]

        uuT = producto_externo(u, u)
        uTu = producto_interno(u, u)
        H_local = np.eye(m)
        for i in range(m):
            for j in range(m):
                H_local[i][j] -= 2 * uuT[i][j] / uTu

        # Construyo H definitiva
        H = np.eye(m)
        for i in range(col, m):
            for j in range(col, m):
                H[i][j] = H_local[i][j]

        R = multiplicar(H, R)
        Q = multiplicar(Q, H)

    return Q, R

#### Householder efficient implementation
Ref: https://www.youtube.com/watch?v=d-yPM-bxREs

In [8]:
def QR_HH_efficient(A, atol=1e-12):
    """
    A una matriz de m x n (m >= n)
    atol la tolerancia con la que se filtran elementos nulos en R.
    Retorna matrices Q y R calculadas con reflexiones de Householder.
    Si la matriz A no cumple m >= n, retorna None.
    """
    m, n = A.shape
    if m < n:
        return None

    Q = np.eye(m)
    A = np.array(A, dtype=np.float64) # Copia modificable de la matriz

    for k in range(n):
        # Construir el reflector de Householder
        z = A[k:m, k]
        v = np.zeros_like(z)
        v[0] = np.sign(z[0]) * norma(z, p=2)
        v += z
        v /= norma(v, p=2)

        # Aplicar la reflexión a cada columna de A y Q
        for j in range(k, n):            
            A[k:m, j] = A[k:m, j] - 2 * producto_interno(v, A[k:m, j]) * v
        for i in range(m):
            Q[i, k:m] = Q[i, k:m] - 2 * producto_interno(v, Q[i, k:m]) * vector_fila(v)

    return Q, A

#### Benchmark

In [11]:
# Tasa de muestreo (30 es el número mágico según alguna literatura)
N = 30

# Tamaño de matriz (para 100 se dispara el tiempo de ejecución)
MATRIZ_N = 30

implementations = {
    'NumPy LAPACK': {'function': np.linalg.qr},
    'Gram-Schmidt naif QR': {'function': QR_GS_naif},
    'Gram-Schmidt efficient QR': {'function': QR_GS_efficient},
    'Householder naif QR': {'function': QR_HH_naif},
    'Householder efficient QR (size-decreasing block updates)': {'function': QR_HH_efficient}
}

for name in implementations:
    implementations[name]['times'] = []
    
for _ in range(N):
    A = np.random.rand(MATRIZ_N, MATRIZ_N)

    for name in implementations:
        function = implementations[name]['function']
        t0 = time.perf_counter()
        function(A)
        t1 = time.perf_counter()

        implementations[name]['times'].append(t1 - t0)

print(f'Matrix size: {MATRIZ_N} x {MATRIZ_N} ({N} samples)')
for name in implementations:
    print(f'## {name}')
    times = implementations[name]['times']
    print(f'Average time: {np.mean(times):.5f} secs')
    print(f'Best time: {np.min(times):.5f} secs')

Matrix size: 30 x 30 (30 samples)
## NumPy LAPACK
Average time: 0.00017 secs
Best time: 0.00013 secs
## Gram-Schmidt naif QR
Average time: 0.04047 secs
Best time: 0.03340 secs
## Gram-Schmidt efficient QR
Average time: 0.01434 secs
Best time: 0.01184 secs
## Householder naif QR
Average time: 1.02379 secs
Best time: 0.85465 secs
## Householder efficient QR (size-decreasing block updates)
Average time: 0.03032 secs
Best time: 0.02463 secs
